In [3]:
import json
import networkx as nx
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import time

# class Node:
#     def __init__(self, address, value):
#         self.address = address
#         self.value = value

load_dotenv()
api_key = os.getenv("covalent_key")
host = "https://api.covalenthq.com"

G = nx.MultiDiGraph()

FTX_ADDRESS = '0x2FAF487A4414Fe77e2327F0bf4AE2a264a776AD2'

df = pd.DataFrame(columns=['tx_hash', 'from_address', 'to_address','transfer_type', 'delta'])  

node_list = []

def get_parent(myaddress, count):

    if(myaddress == FTX_ADDRESS or count >= 5):
        return

    endpoint = "/v1/1/address/{address}/transfers_v2/?contract-address=0x50D1c9771902476076eCFc8B2A83Ad6b9355a4c9&key={API_KEY}"

    print("starting json stuff")
    
    try:
        endpoint = endpoint.format(address = myaddress, API_KEY = api_key)
        newrequest = requests.get(host+endpoint)
        new_json_data = json.loads(newrequest.text)
    
        print("json stuff done")

        mylist = []
        print(count)

        for stuff in new_json_data['data']['items']:
            if(stuff['transfers'][0]['tx_hash'] not in df['tx_hash']):
                
                mylist.append(stuff['transfers'][0]['from_address'])

                df.loc[len(df.index)] = [stuff['transfers'][0]['tx_hash'],stuff['transfers'][0]['from_address'], stuff['transfers'][0]['to_address'], stuff['transfers'][0]['transfer_type'],stuff['transfers'][0]['delta']]

                if(stuff['transfers'][0]['from_address'] not in node_list):
                    node_list.append(stuff['transfers'][0]['from_address'])
                    G.add_node(stuff['transfers'][0]['from_address'])
                
                if(stuff['transfers'][0]['to_address'] not in node_list):
                    node_list.append(stuff['transfers'][0]['to_address'])
                    G.add_node(stuff['transfers'][0]['to_address'])
                    
                G.add_edge(stuff['transfers'][0]['from_address'], stuff['transfers'][0]['to_address'])


        nx.draw(G)
        time.sleep(1)

        for ad in mylist:
            get_parent(ad, count+1)

    except:
        print(f"Json stuff failed as address was {myaddress}")